# `IsotropicGasID`: An Einstein Toolkit Initial Data Thorn for isotripic gas initial data
## Author: Leo Werneck
### Formatting improvements courtesy Brandon Clark

**Notebook Status:** <font color='red'><b> In progress </b></font>

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$ 

This notebook is organized as follows

1. [Step 1](#initialize_nrpy): Initialize core NRPy+/Python modules

1. [Step 3](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='initialize_nrpy'></a>

# Step 1: Initialize core NRPy+/Python modules \[Back to [top](#toc)\]
$$\label{initialize_nrpy}$$

Let us start by loading the core NRPy+ and Python modules that we will need to write down our C code.

In [1]:
# Step 1: Initialize core NRPy+ and Python modules
import os,sys,shutil           # Standard Python modules for multiplatform OS-level functions
import sympy as sp             # SymPy: Symbolic algebra package for Python (NRPy+'s backbone)
sys.path.append(os.path.join("nrpy_core"))
import outputC as outC         # NRPy+: Core C output functions
import NRPy_param_funcs as par # NRPy+: Parameter interface
import cmdline_helper as cmd   # NRPy+: Multi-platform Python command-line interface
import grid as gri             # NRPy+: Numerical grids interface
import indexedexp as ixp       # NRPy+: Indexed expression support

# Step 1.a: Set thorn name
ThornName = "IsotropicGasID"

# Step 1.b: Create the thorn directory
Thorndir = os.path.join(ThornName)
if os.path.exists(Thorndir):
    shutil.rmtree(Thorndir)
cmd.mkdir(Thorndir)

# Step 1.c: Create the source code directory
Srcdir = os.path.join(Thorndir,"src")
cmd.mkdir(Srcdir)

# Step 1.d: Set precision of the code to CCTK_REAL
par.set_parval_from_str("outputC::PRECISION","CCTK_REAL")

# Step 1.e: Set gridfunction memory access to ETK
par.set_parval_from_str("grid::GridFuncMemAccess","ETK")

<a id='id_symbolic'></a>

# Step 2: Symbolic expressions for the initial data \[Back to [top](#toc)\]
$$\label{id_symbolic}$$

We now generate symbolic expressions for the spacetime and hydrodynamics initial data:

$$
\newcommand{\rhob}{\rho_{\rm b}}
\newcommand{\ye}{Y_{\rm e}}
\begin{aligned}
\alpha = 0,\ \beta^{i} = 0&,\ \gamma_{ij} = \delta_{ij},\ k_{ij} = 0\\
\rhob = \left(\rhob\right)_{\rm input},\ \ye &= \left(\ye\right)_{\rm input},\ T = T_{\rm input},\\
v^{i} = 0&,\ B^{i} = 0.
\end{aligned}
$$

To make it easier for users to adopt this thorn, we allow for *any* spacetime/hydrodynamics gridfunctions within the Einstein Toolkit to be used.

In [2]:
# Step 2: Symbolic expressions for the initial data
# Step 2.a: Declare basic constants
zero = sp.sympify(0)
one  = sp.sympify(1)

# Step 2.b: Initialize list of gridfunctions & expressions
vars_list = []
expr_list = []

# Step 2.c: Thorn ID parameters
ID_Ye,ID_rhob,ID_T = par.Cparameters("REAL",ThornName,
                                     [ThornName+"_Ye",ThornName+"_rho",ThornName+"_T"],
                                     [     "-1"      ,      "-1"      ,     "-1"     ])

# Step 2.c: Symbolic expressions for the ID
vars_list.extend(["alpha",  "Ye", "rhob", "T"])
expr_list.extend([  one  ,ID_Ye,ID_rhob,ID_T])
for i in range(3):
    vars_list.append("beta"+chr(ord('x')+i))
    expr_list.append(zero)
    vars_list.append("v"+chr(ord('x')+i))
    expr_list.append(zero)
    vars_list.append("B"+chr(ord('x')+i))
    expr_list.append(zero)
    for j in range(i,3):
        vars_list.append("g"+chr(ord('x')+i)+chr(ord('x')+j))
        if j==i:
            expr_list.append(one)
        else:
            expr_list.append(zero)
        vars_list.append("k"+chr(ord('x')+i)+chr(ord('x')+j))
        expr_list.append(zero)

# Step 2.d: Sort the lists
vars_list,expr_list = [list(x) for x in zip(*sorted(zip(vars_list,expr_list), key=lambda pair: pair[0]))]

# Step 2.e: Generate the ID & gf pointer strings
N = len(vars_list)
ID_string = ""
GF_pointers_string = ""
indent = "        "
for n in range(N):
    gf    = vars_list[n]
    value = expr_list[n]
    ID_string += indent+"ID_"+gf+"[idx] = "+str(value)+";\n"
    GF_pointers_string += """
CCTK_REAL *ID_"""+gf+" = (CCTK_REAL *)(CCTK_VarDataPtr(cctkGH,timelevel,"ThornName+"_"+gf+"""))
if( !ID_"""+gf+r""" ) CCTK_VError (__LINE__, __FILE__, CCTK_THORNSTRING, "Couldn't get data pointer of input array variable '%s'", HamiltonianVarString);"""

# Step 2.f: Get gridfunction pointers string

for n in range(N):
    GF_pointers_string += """

#CCTK_REAL* H_gf   = (CCTK_REAL*)(CCTK_VarDataPtr(cctkGH,timelevel, HamiltonianVarString));
#if(  !H_gf  ) CCTK_VError (__LINE__, __FILE__, CCTK_THORNSTRING, "Couldn't get data pointer of input array variable '%s'", HamiltonianVarString);

<a id='function_registration'></a>

# Step 3: Function registration \[Back to [top](#toc)\]
$$\label{function_registration}$$

We now start writing the `IsotropicGasID`. We follow [Murguia-Berthier *et al.* (2021)](https://arxiv.org/pdf/2106.05356.pdf) and set up an initial condition for a gas with constant density $\rho$, constant temperature $T$, constant electron fraction $Y_{\rm e}$, and initialize the spacetime to [Minkowski space](https://en.wikipedia.org/wiki/Minkowski_space).

<a id='driver_function'></a>

## Step 3.a:  `IsotropicGasID` - the thorn's driver function \[Back to [top](#toc)\]
$$\label{driver_function}$$

Below we implement the thorn's driver function, `IsotropicGasID`, which is responsible for setting up the desired initial data.

In [9]:
def add_to_Cfunction_dict__IsotropicGasID():
    desc = """
(c) 2021 Leo Werneck

This is the thorn's driver function, responsible
for setting the initial data to that of an isotropic
gas of constant density, temperature, and electron
fraction in Minkowski space.
"""
    includes = ["cctk.h","cctk_Arguments.h","cctk_Parameters.h"]
    prefunc  = ""
    c_type   = "void"
    name     = "IsotropicGasID"
    params   = "CCTK_ARGUMENTS"
    body     = r"""
  // Step 1: Get access to gridfunctions and parameters
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;

  // Step 3: Loop over the grid and set the ID
#pragma omp parallel for
  for(int k=0;k<cctk_lsh[2];k++) {
    for(int j=0;j<cctk_lsh[1];j++) {
      for(int i=0;i<cctk_lsh[0];i++) {
      
        const int idx = CCTK_GFINDEX3D(cctkGH,i,j,k);

"""+ID_string+"""
      }
    }
  }
"""
    loopopts = ""
    outC.outCfunction(os.path.join(Srcdir,name+".c"),
                      includes=includes,prefunc=prefunc,desc=desc,c_type=c_type,
                      name=name,params=params,body=body,enableCparameters=False)
#     outC.add_to_Cfunction_dict(includes=includes,prefunc=prefunc,desc=desc,c_type=c_type,
#                                name=name,params=params,body=body,enableCparameters=False)

<a id='latex_pdf_output'></a>

# Step n: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-Tutorial-ETK_thorn-IsotropicGasID.pdf](Tutorial-Tutorial-ETK_thorn-IsotropicGasID.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [5]:
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-Tutorial-ETK_thorn-IsotropicGasID")

Created Tutorial-Tutorial-ETK_thorn-IsotropicGasID.tex, and compiled LaTeX
    file to PDF file Tutorial-Tutorial-ETK_thorn-IsotropicGasID.pdf
